### 임베딩(Embedding)

- 특정 단어, 문장, 문서를 벡터로 변환

In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
client = OpenAI()
# 초창기 모델
# embedding_model = "text-embedding-ada-002" 
# 2024-01 모델
embedding_model = "text-embedding-3-small"

In [3]:
result = client.embeddings.create(input='저는 배가 고파요', model=embedding_model)

In [5]:
# print(result)
print("embedding ", result.data[0].embedding)
print("차원 ", len(result.data[0].embedding))

embedding  [0.017392737790942192, -0.007788157090544701, -0.029023703187704086, 0.007519599981606007, 0.001110239652916789, -0.05871147662401199, -0.03375031054019928, 0.03361358866095543, -0.04668011516332626, -0.015400531701743603, -0.020644720643758774, 0.05128953233361244, -0.0011773789301514626, -0.00381839438341558, -0.01697281189262867, 0.0023633029777556658, -0.058281783014535904, 0.0003994787693955004, 0.005024460144340992, -0.03279326856136322, -0.02459006942808628, -0.015791160985827446, -0.023281462490558624, -0.006777405738830566, 0.0020544622093439102, -0.00045319017954170704, 0.03955114260315895, -0.033672183752059937, 0.003637728514149785, -0.011601668782532215, 0.009824308566749096, -0.04769574850797653, 0.01806657202541828, -0.048398882150650024, -0.019013846293091774, -0.038769885897636414, 0.011679794639348984, -0.03769565746188164, -0.004209022969007492, 0.0011761581990867853, -0.029160423204302788, 0.0015637350734323263, -0.01797867938876152, 0.008950277231633663,

- 벡터 유사도
    - 임베딩으로 구한 벡터 숫자 집합이 나타내는 항목의 유사도

In [6]:
import numpy as np
from numpy.linalg import norm
from numpy import dot

def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [7]:
vec1 = np.array([0,1,1,1])
vec2 = np.array([1,0,1,1])
vec3 = np.array([2,0,2,2])

print('벡터1과 벡터2의 유사도 : ',cos_sim(vec1, vec2))
print('벡터1과 벡터3의 유사도 : ',cos_sim(vec1, vec3))
print('벡터2과 벡터3의 유사도 : ',cos_sim(vec2, vec3))

벡터1과 벡터2의 유사도 :  0.6666666666666667
벡터1과 벡터3의 유사도 :  0.6666666666666667
벡터2과 벡터3의 유사도 :  1.0000000000000002


In [8]:
import pandas as pd

data = [
    '저는 배가 고파요',
    '저기 배가 지나가네요',
    '굶어서 허기가 지네요',
    '허기 워기라는 게임이 있는데 즐거워',
    '스팀에서 재밌는거 해야지',
    '스팀에어프라이어로 연어구이 해먹을거야'
]

df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야


In [9]:
# 임베딩 구하기

def embedding_func(text):
    response = client.embeddings.create(input=text, model=embedding_model)
    result = response.data[0].embedding
    return result

In [10]:
# 데이터 프레임에 함수 적용
df['embedding'] = df.apply(lambda row:embedding_func(row.text), axis=1)
df

,text,embedding
0,저는 배가 고파요,"[0.017392737790942192, -0.007788157090544701, ..."
1,저기 배가 지나가네요,"[0.03943436220288277, 9.303291153628379e-05, -..."
2,굶어서 허기가 지네요,"[0.04150686785578728, 0.024796810001134872, -0..."
3,허기 워기라는 게임이 있는데 즐거워,"[0.007494595367461443, -0.0038305148482322693,..."
4,스팀에서 재밌는거 해야지,"[0.007193937432020903, 0.054016903042793274, -..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[0.050126370042562485, 0.003772211028262973, 0..."


In [11]:
# 코사인 유사도를 이용한 유사한 문장 추출

def return_answer_candidate(df, query):
    query_embedding = embedding_func(query)
    df['similarity'] = df.embedding.apply(lambda x:cos_sim(np.array(x), np.array(query_embedding)))
    top_three_doc = df.sort_values("similarity",ascending=False).head(3)
    return top_three_doc

In [12]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
5,스팀에어프라이어로 연어구이 해먹을거야,"[0.050126370042562485, 0.003772211028262973, 0...",0.312703
0,저는 배가 고파요,"[0.017392737790942192, -0.007788157090544701, ...",0.239211
2,굶어서 허기가 지네요,"[0.04150686785578728, 0.024796810001134872, -0...",0.217988


In [13]:
data = [
    '음식이 맛있었고, 웨이터가 친절했습니다.',
    '분위기는 좋았지만 음식은 그냥 그래요',
    '저는 음식이 마음에 들었고 웨이터는 매우 세심했습니다'
]

df = pd.DataFrame(data, columns=['text'])
df

,text
0,"음식이 맛있었고, 웨이터가 친절했습니다."
1,분위기는 좋았지만 음식은 그냥 그래요
2,저는 음식이 마음에 들었고 웨이터는 매우 세심했습니다


In [14]:
df['embedding'] = df.apply(lambda row:embedding_func(row.text), axis=1)
df

,text,embedding
0,"음식이 맛있었고, 웨이터가 친절했습니다.","[-0.022967981174588203, -0.0227913036942482, -..."
1,분위기는 좋았지만 음식은 그냥 그래요,"[-0.006434813607484102, 0.0012769235763698816,..."
2,저는 음식이 마음에 들었고 웨이터는 매우 세심했습니다,"[-0.02784585766494274, -0.024587729945778847, ..."


In [15]:
print('0번리뷰와 1번리뷰 유사도 : ',cos_sim(df.iloc[0,1], df.iloc[1,1]))
print('0번리뷰와 2번리뷰 유사도 : ',cos_sim(df.iloc[0,1], df.iloc[2,1]))
print('1번리뷰와 2번리뷰 유사도 : ',cos_sim(df.iloc[1,1], df.iloc[2,1]))

0번리뷰와 1번리뷰 유사도 :  0.3933044841925224
0번리뷰와 2번리뷰 유사도 :  0.7514147410194216
1번리뷰와 2번리뷰 유사도 :  0.41026012941469564


In [16]:
df = pd.read_csv("./data/fine_food_reviews_1k_fully_translated_korean.csv", index_col=0)
df.head()

,Time,ProductId,UserId,Score,Summary,Text
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.
2,1351123200,B000JMBE7M,AQX1N6A51QOKG,4,그것은 Blanc Mange는 아니지만 나쁘지 않습니다,커스터드가 계란이없는 커스터드인지 확실하지 않지만 이것은 비건 채식 팬케이크 레시피...
3,1351123200,B004AHGBX4,A2UY46X0OSNVUQ,3,이들은 또한 소금과 바다 소금이 아닙니다,나는 당신이 당신이 무엇을 얻는 지 알 수 있고 뼈 나 어두운 고기가 없다는 사실을...
4,1351123200,B001BORBHO,A1AFOYZ9HSM2CZ,5,제품에 만족합니다,내 개는 가려운 피부로 고통 스러웠습니다.음식과 피부 문제는 몇 주 안에 정리되었습니다.


In [17]:
# Summary + Text 결합
df['combined'] = "Title: "+df['Summary'].str.strip() + '; Content: '+df['Text'].str.strip()

# NaN 제거
df = df.dropna()

In [18]:
df.shape

(999, 7)

In [19]:
import tiktoken

embedding_encoding = "cl100k_base"

encoding = tiktoken.get_encoding(embedding_encoding)
encoding.encode("이것은 한권의 책입니다.")

[13094,
 28740,
 225,
 34804,
 62398,
 166,
 114,
 234,
 21028,
 3396,
 109,
 227,
 80052,
 13]

In [20]:
# text-embedding-3-small : 8,191 최대 입력 토큰 수

max_token = 8000

# combined 컬럼의 내용이 토큰을 얼마나 소모하는지 계산
df['n_tokens'] = df.combined.apply(lambda x:len(encoding.encode(x)))

In [22]:
df.head(1)

# 8000 개를 초과하는 리뷰 제거
df = df[df['n_tokens'] <= max_token]

In [23]:
df.shape

(999, 8)

In [24]:
# 임베딩 구하기
# df['embedding'] = df['combined'].apply(lambda row:embedding_func(row), axis=1)

# 임베딩 결과가 포함된 csv 저장
# df.to_csv("경로/파일명")

# 임베딩 결과가 포함된 csv 가져오기
df = pd.read_csv("./data/fine_food_reviews_with_embeddings_1k.csv", index_col=0)
df.head(3)

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...,87,"[-0.0022805905900895596, 0.0007801640313118696..."
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...,55,"[0.05709123983979225, 0.005073584616184235, -0..."
2,B000JMBE7M,AQX1N6A51QOKG,4,그것은 Blanc Mange는 아니지만 나쁘지 않습니다,커스터드가 계란이없는 커스터드인지 확실하지 않지만 이것은 비건 채식 팬케이크 레시피...,Title: 그것은 Blanc Mange는 아니지만 나쁘지 않습니다; Content...,279,"[0.03149786591529846, -0.02936280518770218, -0..."


In [26]:
# embedding 컬럼이 object(문자열)
df.info()

# 리스트 형태로 변경
df['embedding'] = df['embedding'].apply(eval).to_list()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ProductId  999 non-null    object
 1   UserId     999 non-null    object
 2   Score      999 non-null    int64 
 3   Summary    999 non-null    object
 4   Text       999 non-null    object
 5   combined   999 non-null    object
 6   n_tokens   999 non-null    int64 
 7   embedding  999 non-null    object
dtypes: int64(2), object(6)
memory usage: 70.2+ KB


In [28]:
description = '맛있는 콩'

# description 와 유사한 상위 3개 리뷰 검색
result = return_answer_candidate(df, description)
result[['Summary','Text']]

,Summary,Text
771,자메이카 푸른 콩,로스팅하기위한 우수한 커피 콩 우리 가족은 더 많은 양의 맛을 위해 5 파운드를 더...
88,맛있는,나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍부한 맛을줍니다. 나는 법에있는 어머니...
460,아주 맛있는,아주 좋은 것은 여전히 ​​약간 왼쪽을 가지고 있습니다. 나는 짠 맛이 마음에 든다.
